In [2]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks
/Users/idekeradmin/Dropbox/GitHub/agent_evaluation
['/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks', '/opt/anaconda3/lib/python311.zip', '/opt/anaconda3/lib/python3.11', '/opt/anaconda3/lib/python3.11/lib-dynload', '', '/Users/idekeradmin/.local/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation']


In [8]:
from agent_evaluation.hierarchy import Hierarchy, Network
import json
import ndex2 
from ndex2.cx2 import RawCX2NetworkFactory

# Create NDEx2 python client
client = ndex2.client.Ndex2()

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Download BioGRID: Protein-Protein Interactions (SARS-CoV) from NDEx
# https://www.ndexbio.org/viewer/networks/669f30a3-cee6-11ea-aaef-0ac135e8bacf
# client_resp = client.get_network_as_cx2_stream('669f30a3-cee6-11ea-aaef-0ac135e8bacf')

# Dengue string interactome network c223d6db-b0e2-11ee-8a13-005056ae23aa
client_resp = client.get_network_as_cx2_stream('c223d6db-b0e2-11ee-8a13-005056ae23aa')

# Convert downloaded interactome network to CX2Network object
interactome = factory.get_cx2network(json.loads(client_resp.content))

# Dengue hierarchy
# https://www.ndexbio.org/viewer/networks/59bbb9f1-e029-11ee-9621-005056ae23aa
client_resp = client.get_network_as_cx2_stream('59bbb9f1-e029-11ee-9621-005056ae23aa')

# Convert downloaded interactome network to CX2Network object
hierarchy = factory.get_cx2network(json.loads(client_resp.content))

# Display information about the hierarchy network and output 1st 100 characters of CX2
print('Name: ' + hierarchy.get_name())
print('Number of nodes: ' + str(len(hierarchy.get_nodes())))
print('Number of nodes: ' + str(len(hierarchy.get_edges())))

# Display information about the interactome network 
print('Name: ' + interactome.get_name())
print('Number of nodes: ' + str(len(interactome.get_nodes())))
print('Number of nodes: ' + str(len(interactome.get_edges())))
      




Name: Dengue model - hidef string 12.0 0.7 (GPT-4 annotated) - L2R
Number of nodes: 203
Number of nodes: 249
Name: dengue string 12.0 0.7
Number of nodes: 1375
Number of nodes: 2792


KeyError: 0

In [11]:
print(json.dumps(list(hierarchy.get_nodes().values())[0], indent=2))

{
  "id": 5971034,
  "v": {
    "CD_Labeled": true,
    "CD_AnnotatedAlgorithm": "Annotated by gProfiler [Docker: coleslawndex/cdgprofilergenestoterm:0.3.0] {{--organism=hsapiens, --maxpval=0.00001, --minoverlap=0.05, --maxgenelistsize=5000}} via CyCommunityDetection Cytoscape App (1.10.0-SNAPSHOT)",
    "name": "C4493669",
    "CommunityDetectionTally::viral_interaction": 73,
    "CD_AnnotatedMembers_Pvalue": 6.248363995620866e-140,
    "CD_AnnotatedMembers_Size": 212,
    "CD_AnnotatedMembers_Overlap": 0.095,
    "CommunityDetectionTally::has_protein_24hr": 23,
    "CD_MemberList_LogSize": 9.827,
    "CommunityDetectionTally::has_protein_48hr": 56,
    "CommunityDetectionTally::has_siRNA": 464,
    "selected": true,
    "CD_MemberList": "A2M ABCD3 ABHD12 ABHD16A ABT1 ACACB ACOT1 ACOX2 ACSS1 ACTR5 ACY1 ADAMTS2 ADAMTS5 ADAMTSL3 ADCY1 ADCY2 ADRB2 AFF3 AFMID AGL AGO1 AGPAT5 AHNAK AHNAK2 AIM2 AKAP14 AKAP5 AKAP7 AKT1S1 ALDH3A1 ALKBH1 ALKBH5 ALOX12 AMDHD2 AMOT AMOTL2 AMPH ANAPC13 ANGPT4 ANK

In [ ]:
from agent_evaluation.llm import ServerModel_LLM

# mixtral7b = ServerModel_LLM("mixtral:instruct", url="https://ollama-api.ideker.ucsd.edu/api/generate", key="Y0hWaWRtbHphWFJ2Y2pveFJHVmhNRFZJTW5BeGJWQXhNRTB3VEZVPQ==")
mixtral7b = ServerModel_LLM("mixtral:instruct", url="https://api.llm.ideker.ucsd.edu/api/chat", key="Y0hWaWRtbHphWFJ2Y2pveFJHVmhNRFZJTW5BeGJWQXhNRTB3VEZVPQ==")

mixtral7b.query("you are a helpful assistant", "what is the best way to make a cup of coffee?")     

Model: mixtral:instruct, Temperature: 0, Max Tokens: 1000, Seed: 42
The request failed with status code: 401


(None, 'The request failed with status code: 401')

## Hierarchy Class

In [16]:
hierarchy.add_network_attribute("experiment_description", "in this experiment, cells were infected with dengue virus and 'omics and phenotypic screening data  was measured", "string")

In [84]:
from agent_evaluation.dataset import Dataset

class Hierarchy():
    def __init__(self, hierarchy_cx, derived_from_cx=None):
        self.hierarchy_cx = hierarchy_cx
        self.derived_from_cx = derived_from_cx

    def get_experiment_description(self):
        return self.hierarchy_cx.get_network_attributes().get("experiment_description")
    
    def get_assemblies(self, filter=None):
        assemblies = []
        for assembly in self.hierarchy_cx.get_nodes().values():
            attributes = assembly.get("v")
            print(attributes.get("name"))
            size = attributes.get("CD_MemberList_Size")
           
            members = attributes.get("CD_MemberList")
            # print(members)
            if members is not None and size is not None:
                if filter is not None:
                    names = filter.get("names")
                    if names is not None and not names.contains(assembly.get_name()):
                        continue
                    max_size = filter.get("max_size")
                    if max_size is not None and size > max_size:
                        print(f'size {size} is greater than max_size {max_size}')
                        continue
                    min_size = filter.get("min_size")
                    if min_size is not None and size < min_size:
                        continue
                assemblies.append(assembly) 
        return assemblies

    def get_datasets(self, filter=None, member_attributes=None, member_data_only=True):
        assemblies = self.get_assemblies(filter)
        print(f'number of assemblies = {len(assemblies)}')        
        datasets = []
        interactome_nodes = self.derived_from_cx.get_nodes().values()
        interactome_data =[]
        for interactome_node in interactome_nodes:
            interactome_node_attributes = interactome_node.get("v")
            if member_attributes is not None:
                interactome_data.append( {key: value for key, value in interactome_node_attributes.items() if key in member_attributes})
            else:
                interactome_data.append(interactome_node_attributes)
        for assembly in assemblies:
            attributes = assembly.get("v")
            members = attributes.get("CD_MemberList").split(" ")
            if member_data_only:
                filtered_interactome_data = [data for data in interactome_data if data['name'] in members]
                datasets.append(Dataset(members, filtered_interactome_data, self.get_experiment_description()))
            else:
                datasets.append(Dataset(members, interactome_data, self.get_experiment_description()))
        return datasets

In [67]:
interactome.get_attribute_declarations()

{'nodes': {'has_rnaSeq_48hr': {'d': 'integer'},
  'has_rnaSeq_24hr': {'d': 'integer'},
  'Condition_48hpi': {'d': 'string'},
  'type': {'v': 'protein', 'd': 'string'},
  'GeneSymbol_48hpi': {'d': 'string'},
  'represents': {'a': 'r', 'd': 'string'},
  'has_protein_24hr': {'d': 'integer'},
  'has_protein_48hr': {'d': 'integer'},
  'UniprotID': {'d': 'string'},
  'siRNA_GeneSymbol': {'d': 'string'},
  'alias': {'d': 'list_of_string'},
  'PPI_GeneSymbol': {'d': 'string'},
  'dengue_MiST_list': {'d': 'string'},
  'GeneSymbol': {'d': 'string'},
  'Condition': {'d': 'string'},
  'viral_interaction': {'d': 'integer'},
  'DV3_24h-Mock_24h': {'d': 'double'},
  'log2FC_48hpi': {'d': 'double'},
  'has_siRNA': {'d': 'integer'},
  'dengue_protein_list': {'d': 'string'},
  'siRNA_Screen_Average_Zscore': {'d': 'double'},
  'name': {'a': 'n', 'd': 'string'},
  'GeneID': {'d': 'integer'},
  'log2FC': {'d': 'double'},
  'DV3_48h-Mock_48h': {'d': 'double'}},
 'networkAttributes': {'disease': {'d': 'strin

In [88]:
dengue_hierarchy = Hierarchy(hierarchy, interactome)
print(dengue_hierarchy.get_experiment_description())
datasets = dengue_hierarchy.get_datasets(member_attributes=["name", "GeneSymbol", "GeneID", "log2FC_48hpi", "pvalue_48hpi", "log2FC_72hpi", "pvalue_72hpi"],
                                         filter={"max_size": 4})
datasets


in this experiment, cells were infected with dengue virus and 'omics and phenotypic screening data  was measured
C4493669
size 908 is greater than max_size 4
C4493670
size 445 is greater than max_size 4
C4493671
size 197 is greater than max_size 4
C4493672
size 193 is greater than max_size 4
C4493673
size 140 is greater than max_size 4
C4493674
size 126 is greater than max_size 4
C4493675
size 96 is greater than max_size 4
C4493676
size 81 is greater than max_size 4
C4493677
size 75 is greater than max_size 4
C4493678
size 59 is greater than max_size 4
C4493679
size 58 is greater than max_size 4
C4493680
size 53 is greater than max_size 4
C4493681
size 48 is greater than max_size 4
C4493682
size 45 is greater than max_size 4
C4493683
size 44 is greater than max_size 4
C4493684
size 39 is greater than max_size 4
C4493685
size 39 is greater than max_size 4
C4493686
size 38 is greater than max_size 4
C4493687
size 38 is greater than max_size 4
C4493688
size 36 is greater than max_size 4
C

In [86]:
datasets[75].data

[{'name': 'HSF1', 'GeneSymbol': 'HSF1', 'GeneID': 3297},
 {'name': 'CDC37', 'GeneSymbol': 'CDC37', 'GeneID': 11140},
 {'name': 'HSP90AA1', 'GeneSymbol': 'HSPCA', 'GeneID': 3320},
 {'name': 'HSPA1B', 'log2FC_48hpi': 3.43733255125146, 'GeneID': 3304},
 {'name': 'CYP2A6', 'GeneSymbol': 'CYP2A6', 'GeneID': 1548},
 {'name': 'NDUFAF6', 'GeneSymbol': 'MGC40214', 'GeneID': 137682},
 {'name': 'TMEM126B', 'GeneSymbol': 'HT007', 'GeneID': 55863},
 {'name': 'SEMA3F', 'GeneSymbol': 'SEMA3F', 'GeneID': 6405},
 {'name': 'PRKACA', 'GeneID': 5566},
 {'name': 'KCNJ1', 'log2FC_48hpi': -2.83222781216424, 'GeneID': 3758},
 {'name': 'FLNA', 'log2FC_48hpi': -2.58851284862048, 'GeneID': 2316},
 {'name': 'PRKAA1', 'GeneID': 5562},
 {'name': 'YWHAZ', 'GeneSymbol': 'YWHAZ', 'GeneID': 7534},
 {'name': 'PDK4', 'log2FC_48hpi': -2.36692831584751, 'GeneID': 5166},
 {'name': 'CDHR2', 'GeneSymbol': 'PCLKC', 'GeneID': 54825},
 {'name': 'USH1G', 'GeneSymbol': 'USH1G', 'GeneID': 124590},
 {'name': 'MAPK9', 'GeneID': 5601}